In [1]:
from stellargraph import datasets
from stellargraph import IndexedArray
from stellargraph import StellarGraph
import pandas as pd

In [2]:
#命名可以没有顺序
author = IndexedArray(index=["a1","a2","a3","a4"])
paper = IndexedArray(index=["p1", "p2", "p3"])
venue = IndexedArray(index=["v1", "v2", "v3"])

In [4]:
edges_AP = pd.DataFrame(
    {"source": ["a1","a2","a3","a4"], "target": ["p1", "p2", "p3", "p3"]},index=[0,1,2,3]
)
edges_PV = pd.DataFrame(
    {"source": ["p1", "p2", "p3"], "target": ["v1", "v2", "v3"]},index=[4,5,6]
)
#index的用处是为每一条边给一个唯一id

In [5]:
Graph = StellarGraph(nodes={"author": author, "paper": paper,"venue": venue},
                     edges={"AP":edges_AP, "PV":edges_PV})
print(Graph.info())

StellarGraph: Undirected multigraph
 Nodes: 10, Edges: 7

 Node types:
  author: [4]
    Features: none
    Edge types: author-AP->paper
  venue: [3]
    Features: none
    Edge types: venue-PV->paper
  paper: [3]
    Features: none
    Edge types: paper-AP->author, paper-PV->venue

 Edge types:
    author-AP->paper: [4]
        Weights: all 1 (default)
        Features: none
    paper-PV->venue: [3]
        Weights: all 1 (default)
        Features: none


In [ ]:
#graph save


#graph load

# Metapath2Vec

In [6]:
walk_length = 100  # maximum length of a random walk to use throughout this notebook

# specify the metapath schemas as a list of lists of node types.
metapaths = [
    ["author", "paper", "author"],
    ["author", "paper", "paper", "author"],
    ["author", "paper", "venue","paper", "author"],
]

In [8]:
from stellargraph.data import UniformRandomMetaPathWalk

# Create the random walker
rw = UniformRandomMetaPathWalk(Graph)

walks = rw.run(
    nodes=list(Graph.nodes()),  # root nodes
    length=walk_length,  # maximum length of a random walk
    n=1,  # number of random walks per root node
    metapaths=metapaths,  # the metapaths
)

print("Number of random walks: {}".format(len(walks)))

Number of random walks: 12


In [14]:
from gensim.models import Word2Vec

model = Word2Vec(walks, vector_size=64, window=5, min_count=0, sg=1, workers=2, epochs=100)
model.save('word2vec.model')
#model = Word2Vec.load('word2vec.model')

In [15]:
vectors = model.wv.vectors #获取模型中全部的词向量
vectors.shape

(10, 64)

In [18]:
words = model.wv.index_to_key
words

['p3', 'p2', 'p1', 'a4', 'a2', 'a1', 'a3', 'v3', 'v2', 'v1']

In [21]:
vecs=model.wv['a1']
vecs

array([-0.033222  ,  0.18314068,  0.821272  ,  0.3406464 , -0.42332375,
       -0.07839388,  0.416241  ,  0.1398861 ,  0.39403313,  0.14519005,
        0.21091036,  0.16512315, -0.09368457, -0.3365461 , -0.02940113,
        0.5316532 ,  0.2649583 , -0.10001535, -0.17108014,  0.43290895,
        0.16123572,  0.19841623,  0.5654438 , -0.50825125,  0.41142783,
        0.21390761,  0.14099912,  0.15759242,  0.13340913,  0.1553423 ,
        0.25670564,  0.29594412, -0.8045267 , -0.331459  , -0.38763082,
       -0.2227925 ,  0.1884977 ,  0.75423473,  0.40181893, -0.4412597 ,
        0.29801166,  0.524854  ,  0.25755912, -0.4073466 ,  0.4461672 ,
       -0.10353985,  0.03999948,  0.22882785, -0.36125726, -0.5556171 ,
        0.40251294, -0.42223337, -0.4195769 ,  0.5935092 ,  0.05658446,
       -0.37609524, -0.24926893,  0.38941234, -0.4057807 , -0.19100171,
       -0.31339303,  0.08781558,  0.27605128, -0.05178364], dtype=float32)